This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/blob/wallaroo2024.1_tutorials/wallaroo-run-anywhere/edge-architecture-publish).

## Run Anywhere for ARM Architecture Tutorial: Preparation

Wallaroo Run Anywhere provides model deployment in any device, any cloud, and any architecture.  Models uploaded to Wallaroo are set to their targeted architecture.  By default, this is `X64`.

The model's deployment configuration is tied to its architecture settings.  That deployment configuration is carried over to the models publication in Open Container Initiative (OCI) Registries, which allows edge model deployments on X64 and ARM architectures.

This tutorial demonstrates deploying a ML model trained to predict house prices to X64 and ARM edge locations through the following steps.

* Setting up a workspace, and pipeline.
* Upload a model set to the X64 architecture, and the same model set to the ARM architecture.
* Performing a sample set of inferences to verify the deployment.
* Publish the deployed model to an Open Container Initiative (OCI) Registry for both X64 and ARM deployments.
* Deploy the model to both X64 and ARM edge devices.
* Perform similar inferences on the edge devices for each architecture and show the results.

In this notebook, we use a ONNX model pre-trained to predict house prices for our examples.

## Goal

Model insights monitors the output of the house price model over a designated time window and compares it to an expected baseline distribution. We measure the performance of model deployments in different locations and compare that to the baseline to detect model drift.

### Resources

This tutorial provides the following:

* Models:
  * `models/rf_model.onnx`: The champion model trained to predict house prices.
  * Various inputs:
    * `smallinputs.df.json`: A set of house inputs that tends to generate low house price values.
    * `biginputs.df.json`: A set of house inputs that tends to generate high house price values.
    * `normal-inputs.df.json`: A set of house inputs with a range of house values.

### Prerequisites

* A deployed Wallaroo instance with [Edge Registry Services](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-edge-deployment/#enable-wallaroo-edge-deployment-registry) and [Edge Observability enabled](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-edge-deployment/#set-edge-observability-service).
* The following Python libraries installed:
  * [`wallaroo`](https://pypi.org/project/wallaroo/): The Wallaroo SDK. Included with the Wallaroo JupyterHub service by default.
  * [`pandas`](https://pypi.org/project/pandas/): Pandas, mainly used for Pandas DataFrame
  * `json`: Used for format input data for inference requests.
* A X64 Docker deployment to deploy the model on an edge location.


## Steps

* Upload the house price model twice:  The first model version sets the model to the `X64` architecture, the second version is set to the `ARM` architecture.
* Create two pipeline versions:
  * The first pipeline version uses the model with the `X64` targeted architecture.  This demonstrates that the pipeline deployment configuration inherits the model's `X64` setting.
  * The first pipeline version uses the model with the `ARM` targeted architecture.  This demonstrates that the pipeline deployment configuration inherits the model's `ARM` setting.
* Publish both pipeline versions to an OCI registry.
* Deploy the models from the pipeline versions to their respective architecture.
* Perform sample inferences on both deployed model architectures.

### Import Libraries

The first step will be to import our libraries, and set variables used through this tutorial.

In [1]:
import wallaroo
from wallaroo.object import EntityNotFoundError
from wallaroo.framework import Framework
from wallaroo.engine_config import Architecture

from IPython.display import display

# used to display DataFrame information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

import datetime
import time

workspace_name = f'run-anywhere-house-price-architecture-demonstration-tutorial'
main_pipeline_name = f'architecture-demonstration'
model_name_control = f'house-price-estimator'
model_file_name_control = './models/rf_model.onnx'

# ignoring warnings for demonstration
import warnings
warnings.filterwarnings('ignore')

# used to display DataFrame information without truncating
from IPython.display import display
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [2]:
def get_workspace(name, client):
    workspace = None
    for ws in client.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = client.create_workspace(name)
    return workspace

### Connect to the Wallaroo Instance

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

If logging into the Wallaroo instance through the internal JupyterHub service, use `wl = wallaroo.Client()`.  For more information on Wallaroo Client settings, see the [Client Connection guide](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-client/).

In [6]:
# Login through local Wallaroo instance

wl = wallaroo.Client()

### Create Workspace

We will create a workspace to manage our pipeline and models.  The following variables will set the name of our sample workspace then set it as the current workspace.

Workspace, pipeline, and model names should be unique to each user, so we'll add in a randomly generated suffix so multiple people can run this tutorial in a Wallaroo instance without effecting each other.

In [4]:
workspace = get_workspace(workspace_name, wl)

wl.set_current_workspace(workspace)

{'name': 'run-anywhere-house-price-architecture-demonstration-tutorial', 'id': 62, 'archived': False, 'created_by': 'b4a9aa3d-83fc-407a-b4eb-37796e96f1ef', 'created_at': '2024-03-04T19:46:11.016316+00:00', 'models': [], 'pipelines': []}

### Upload the Model with Targeted Architecture

For our example, we will upload the champion model that has been trained to derive house prices from a variety of inputs.  The model file is `rf_model.onnx`, and is uploaded with the name `house-price-estimator`.

Models are uploaded to Wallaroo via the [`wallaroo.client.upload_model`](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-reference-guide/client/#Client.upload_model) method which takes the following arguments:

| Parameter | Type | Description |
|---|---|---|
| **path** | *String* (*Required*) | The file path to the model. |
| **framework** | *wallaroo.framework.Framework* (*Required*) | The model's framework.  See [Wallaroo SDK Essentials Guide: Model Uploads and Registrations](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-model-uploads/) for supported model frameworks. |
| **input_schema** | *pyarrow.lib.Schema* (*Optional*)  | The model's input schema.  **Only required for non-Native Wallaroo frameworks.  See [Wallaroo SDK Essentials Guide: Model Uploads and Registrations](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-model-uploads/) for more details. |
| **output_schema** | *pyarrow.lib.Schema* (*Optional*)  | The model's output schema.  **Only required for non-Native Wallaroo frameworks.  See [Wallaroo SDK Essentials Guide: Model Uploads and Registrations](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-model-uploads/) for more details. |
| **convert_wait** | *bool* (*Optional*)  | Whether to wait in the SDK session to complete the auto-packaging process for non-native Wallaroo frameworks. |
| **arch** | *wallaroo.engine_config.Architecture* (*Optional*)  | The targeted architecture for the model.  Options are <ol><li>`X86` (*Default*)</li><li>`ARM`</li></ol> |

We upload two versions of of the model:  the first set to the `X86` architecture, the second to the `ARM` architecture.

In [7]:
housing_model_control_x64 = (wl.upload_model(model_name_control, 
                                        model_file_name_control, 
                                        framework=Framework.ONNX,
                                        arch=Architecture.X86)
                                        .configure(tensor_fields=["tensor"])
                        )

housing_model_control_arm = (wl.upload_model(model_name_control, 
                                        model_file_name_control, 
                                        framework=Framework.ONNX,
                                        arch=Architecture.ARM)
                                        .configure(tensor_fields=["tensor"])
                        )

In [8]:
display(housing_model_control_x64)

Name,house-price-estimator
Version,f10d93ab-6d64-4714-9372-01898f7bd189
File Name,rf_model.onnx
SHA,e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6
Status,ready
Image Path,None
Architecture,x86
Acceleration,None
Updated At,2024-04-Mar 19:46:41


In [9]:
display(housing_model_control_arm)

Name,house-price-estimator
Version,a925689e-6b43-49a7-8023-c5efdffcfdcd
File Name,rf_model.onnx
SHA,e22a0831aafd9917f3cc87a15ed267797f80e2afa12ad7d8810ca58f173b8cc6
Status,ready
Image Path,None
Architecture,arm
Acceleration,None
Updated At,2024-04-Mar 19:46:41


### Build the Pipeline

This pipeline is made to be an example of an existing situation where a model is deployed and being used for inferences in a production environment.

We will create two versions of the pipeline:

* One version with model version set to the `X86` architecture.
* The second version with the model version set to the `ARM` architecture.

In both cases, the **model's targeted architecture** will be inherited by the pipeline version; no additional pipeline settings are required for that architecture.  Each will have a deployment configuration set to 1 replica, 1 CPU, and 1 Gb of RAM.

In [12]:
mainpipeline = wl.build_pipeline(main_pipeline_name)
# clear the steps if used before
mainpipeline.clear()

# set the model step with the x86 targeted model
mainpipeline.add_model_step(housing_model_control_x64)

mainpipeline_x86_version = mainpipeline.create_version()

# clear the previous steps
mainpipeline.clear()

# set the model step with the ARM targeted model
mainpipeline.add_model_step(housing_model_control_arm)

mainpipeline_arm_version = mainpipeline.create_version()

#minimum deployment config
deploy_config = wallaroo.DeploymentConfigBuilder().replica_count(1).cpus(1).memory("1Gi").build()

In [13]:
display(mainpipeline_x86_version)

name,architecture-demonstration
version,996fcb52-06fe-43d8-9000-31d6575dbbfa
creation_time,2024-04-Mar 19:47:14
last_updated_time,2024-04-Mar 19:47:14
deployed,False
tags,
steps,house-price-estimator


In [14]:
display(mainpipeline_arm_version)

name,architecture-demonstration
version,83eb1f43-1ed5-4524-b1b0-138cc0861306
creation_time,2024-04-Mar 19:47:14
last_updated_time,2024-04-Mar 19:47:14
deployed,False
tags,
steps,house-price-estimator


## Edge Deployment

We now deploy the pipeline versions to our edge devices.

* Publish the pipeline versions:  Publishes the pipeline versions to the OCI registry with the deployment configuration inherited from the pipeline versions, with the architecture settings inherited from the model versions.
* Deploy Edge:  Deploy the edge device with the edge location settings.

### Publish Pipeline Versions

Publishing the pipeline uses the pipeline `wallaroo.pipeline.publish()` command.  This requires that the Wallaroo Ops instance have [Edge Registry Services](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-edge-deployment/#enable-wallaroo-edge-deployment-registry) enabled.

The following publishes the pipeline to the OCI registry and displays the container details.  For more information, see [Wallaroo SDK Essentials Guide: Pipeline Edge Publication](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-sdk-guides/wallaroo-sdk-essentials-guide/wallaroo-sdk-essentials-pipelines/wallaroo-sdk-essentials-pipeline-publication/).

In [15]:
assay_pub_x86 = mainpipeline_x86_version.publish()
assay_pub_x86

Waiting for pipeline publish... It may take up to 600 sec.
Pipeline is Publishing.....Published.


ID,16
Pipeline Name,architecture-demonstration
Pipeline Version,996fcb52-06fe-43d8-9000-31d6575dbbfa
Status,Published
Engine URL,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2024.1.0-main-4609
Pipeline URL,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/architecture-demonstration:996fcb52-06fe-43d8-9000-31d6575dbbfa
Helm Chart URL,oci://us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/charts/architecture-demonstration
Helm Chart Reference,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/charts@sha256:56ee83fedc6b52ceb1f0d73e4ac45575a01686855ecae9cb7509a84f36cb3ec7
Helm Chart Version,0.0.1-996fcb52-06fe-43d8-9000-31d6575dbbfa
Engine Config,"{'engine': {'resources': {'limits': {'cpu': 4.0, 'memory': '3Gi'}, 'requests': {'cpu': 4.0, 'memory': '3Gi'}, 'accel': 'none', 'arch': 'x86', 'gpu': False}}, 'engineAux': {'autoscale': {'type': 'none'}, 'images': None}, 'enginelb': {'resources': {'limits': {'cpu': 1.0, 'memory': '512Mi'}, 'requests': {'cpu': 0.2, 'memory': '512Mi'}, 'accel': 'none', 'arch': 'x86', 'gpu': False}}}"
User Images,[]


In [17]:
print(assay_pub_x86)

PipelinePublish(created_at=datetime.datetime(2024, 3, 4, 19, 47, 21, 895583, tzinfo=tzutc()), docker_run_variables={}, engine_config={'engine': {'resources': {'limits': {'cpu': 4.0, 'memory': '3Gi'}, 'requests': {'cpu': 4.0, 'memory': '3Gi'}, 'accel': 'none', 'arch': 'x86', 'gpu': False}}, 'engineAux': {'autoscale': {'type': 'none'}, 'images': None}, 'enginelb': {'resources': {'limits': {'cpu': 1.0, 'memory': '512Mi'}, 'requests': {'cpu': 0.2, 'memory': '512Mi'}, 'accel': 'none', 'arch': 'x86', 'gpu': False}}}, id=16, pipeline_name='architecture-demonstration', pipeline_version_id=121, replaces=[], status='Published', updated_at=datetime.datetime(2024, 3, 4, 19, 47, 21, 895583, tzinfo=tzutc()), user_images=[], created_by='b4a9aa3d-83fc-407a-b4eb-37796e96f1ef', edge_bundles=<wallaroo.wallaroo_ml_ops_api_client.types.Unset object at 0x7a5881331d30>, engine_url='us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2024.1.0-main-4609'

In [16]:
assay_pub_arm = mainpipeline_arm_version.publish()
assay_pub_arm

Waiting for pipeline publish... It may take up to 600 sec.
Pipeline is Publishing.....Published.


ID,17
Pipeline Name,architecture-demonstration
Pipeline Version,83eb1f43-1ed5-4524-b1b0-138cc0861306
Status,Published
Engine URL,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2024.1.0-main-4609
Pipeline URL,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/architecture-demonstration:83eb1f43-1ed5-4524-b1b0-138cc0861306
Helm Chart URL,oci://us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/charts/architecture-demonstration
Helm Chart Reference,us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/charts@sha256:1b949ad63d61c6deaec155e6227de478e0e89b3b530e9a7e424a525a967c4691
Helm Chart Version,0.0.1-83eb1f43-1ed5-4524-b1b0-138cc0861306
Engine Config,"{'engine': {'resources': {'limits': {'cpu': 4.0, 'memory': '3Gi'}, 'requests': {'cpu': 4.0, 'memory': '3Gi'}, 'accel': 'none', 'arch': 'x86', 'gpu': False}}, 'engineAux': {'autoscale': {'type': 'none'}, 'images': None}, 'enginelb': {'resources': {'limits': {'cpu': 1.0, 'memory': '512Mi'}, 'requests': {'cpu': 0.2, 'memory': '512Mi'}, 'accel': 'none', 'arch': 'x86', 'gpu': False}}}"
User Images,[]


### DevOps Deployment

The edge deployment is performed with `docker run`, `docker compose`, or `helm` installations.  The following command generates the `docker run` command, with the following values provided by the DevOps Engineer:

* `$REGISTRYURL`: The URL of the OCI registry service hosting the pipeline publish.
* `$OCI_USERNAME`: The username for the OCI registry service.
* `$OCI_PASSWORD`: The password or token for the OCI registry service.

For more details on model edge deployments with Wallaroo, see [Model Operations: Run Anywhere](https://staging.docs.wallaroo.ai/wallaroo-model-operations/wallaroo-model-operations-run-anywhere/).

In [18]:
# create docker run 

docker_command = f'''
docker run -p 8080:8080 \\
    -e DEBUG=true \\
    -e OCI_REGISTRY=$REGISTRYURL \\
    -e CONFIG_CPUS=1 \\
    -e OCI_USERNAME=$OCI_USERNAME \\
    -e OCI_PASSWORD=$OCI_PASSWORD \\
    -e PIPELINE_URL={assay_pub_x86.pipeline_url} \\
    {assay_pub_x86.engine_url}
'''

print(docker_command)


docker run -p 8080:8080 \
    -e DEBUG=true \
    -e OCI_REGISTRY=$REGISTRYURL \
    -e CONFIG_CPUS=1 \
    -e OCI_USERNAME=$OCI_USERNAME \
    -e OCI_PASSWORD=$OCI_PASSWORD \
    -e PIPELINE_URL=us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/architecture-demonstration:996fcb52-06fe-43d8-9000-31d6575dbbfa \
    us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2024.1.0-main-4609



In [19]:
# create docker run 

docker_command = f'''
docker run -p 8080:8080 \\
    -e DEBUG=true \\
    -e OCI_REGISTRY=$REGISTRYURL \\
    -e CONFIG_CPUS=1 \\
    -e OCI_USERNAME=$OCI_USERNAME \\
    -e OCI_PASSWORD=$OCI_PASSWORD \\
    -e PIPELINE_URL={assay_pub_arm.pipeline_url} \\
    {assay_pub_arm.engine_url}
'''

print(docker_command)


docker run -p 8080:8080 \
    -e DEBUG=true \
    -e OCI_REGISTRY=$REGISTRYURL \
    -e CONFIG_CPUS=1 \
    -e OCI_USERNAME=$OCI_USERNAME \
    -e OCI_PASSWORD=$OCI_PASSWORD \
    -e PIPELINE_URL=us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/pipelines/architecture-demonstration:83eb1f43-1ed5-4524-b1b0-138cc0861306 \
    us-central1-docker.pkg.dev/wallaroo-dev-253816/uat/engines/proxy/wallaroo/ghcr.io/wallaroolabs/fitzroy-mini:v2024.1.0-main-4609



### Edge Inference Examples

The following examples demonstrate performing inferences on each of the model deployments on edge devices in different architectures.  For this example, `HOSTNAME_X86` represents the hostname for the X86 architecture edge deployment, and `HOSTNAME_ARM` is the hostname for the ARM architecture edge deployment.  Update these variables to match your deployments.

In [ ]:
HOSTNAME_X86 = 'testboy.local'
HOSTNAME_ARM = 'localhost'

In [ ]:
# generate a set of normal house values from the x86 deployment
!curl -X POST {HOSTNAME_X86}:8080/pipelines/{main_pipeline_name} \
    -H "Content-Type: Content-Type: application/json; format=pandas-records" \
    --data @./data/normal-inputs.df.json

In [ ]:
# generate a set of normal house values from the ARM deployment
!curl -X POST {HOSTNAME_ARM}:8080/pipelines/{main_pipeline_name} \
    -H "Content-Type: Content-Type: application/json; format=pandas-records" \
    --data @./data/normal-inputs.df.json